## Importing the required libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = open('intents.json').read()

In [3]:
intents = json.loads(data)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

## Data Prepprocessing

In [5]:
words=[]
documents=[]
classes=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words

['Hi',
 'there',
 'How',
 'are',
 'you',
 'Is',
 'anyone',
 'there',
 '?',
 'Hey',
 'Hola',
 'Hello',
 'Good',
 'day',
 'Bye',
 'See',
 'you',
 'later',
 'Goodbye',
 'Nice',
 'chatting',
 'to',
 'you',
 ',',
 'bye',
 'Till',
 'next',
 'time',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Awesome',
 ',',
 'thanks',
 'Thanks',
 'for',
 'helping',
 'me',
 'How',
 'you',
 'could',
 'help',
 'me',
 '?',
 'What',
 'you',
 'can',
 'do',
 '?',
 'What',
 'help',
 'you',
 'provide',
 '?',
 'How',
 'you',
 'can',
 'be',
 'helpful',
 '?',
 'What',
 'support',
 'is',
 'offered',
 'How',
 'to',
 'check',
 'Adverse',
 'drug',
 'reaction',
 '?',
 'Open',
 'adverse',
 'drugs',
 'module',
 'Give',
 'me',
 'a',
 'list',
 'of',
 'drugs',
 'causing',
 'adverse',
 'behavior',
 'List',
 'all',
 'drugs',
 'suitable',
 'for',
 'patient',
 'with',
 'adverse',
 'reaction',
 'Which',
 'drugs',
 'dont',
 'have',
 'adverse',
 'reaction',
 '?',
 'Open',
 'blood',
 'pressure',
 'module',
 'Task',
 'relat

In [7]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['How', 'you', 'could', 'help', 'me', '?'], 'options'),
 (['What', 'you', 'can', 'do', '?'], 'options'),
 (['What', 'help', 'you', 'provide', '?'], 'options'),
 (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'),
 (['What', 'support', 'is', 'offered'], 'options'),
 (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'),
 (['Open', 'adverse', 'drugs', 'module'], 'adver

In [8]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [9]:
stop_words=['?', '!']
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in stop_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
# print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


## Train-Test Split

In [10]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Training the model

In [26]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
10/10 [==============================] - 3s 9ms/step - loss: 2.2379 - accuracy: 0.0638
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1551 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1356 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0083 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0084 - accuracy: 0.3191
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8833 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7268 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6216 - accuracy: 0.5106
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4676 - accuracy: 0.6170
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2941 - accuracy: 0.6383
Epoch 11/

10/10 [==============================] - 0s 3ms/step - loss: 0.0619 - accuracy: 0.9787
Epoch 83/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0553 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0911 - accuracy: 0.9574
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0698 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0299 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0260 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0200 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0576 - accuracy: 1.0000
Epoch 92/200


10/10 [==============================] - 0s 3ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0188 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0386 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0328 - accuracy: 0.9787
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0205 - accuracy: 0.9787
Epoc